# This entire notebook is a reimplementation of Andrej's YouTube
[Let's build GPT: from scratch, in code, spelled out.](https://www.youtube.com/watch?v=kCc8FmEb1nY&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=8)

**bold text**## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [1]:
print('Test')

Test


In [2]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# import os
# os.environ['DYLD_LIBRARY_PATH'] = '/usr/local/opt/libunistring/lib:' + os.environ.get('DYLD_LIBRARY_PATH', '')
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-11-24 14:22:05--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8002::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  2.18MB/s    in 0.5s    

2024-11-24 14:22:06 (2.18 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [6]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [8]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [9]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string


print(type(encode))
print(type(decode))

print(encode("hii there"))
print(decode(encode("hii there")))

<class 'function'>
<class 'function'>
[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [10]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.type)
print(data[:1000])

torch.Size([1115394]) <built-in method type of Tensor object at 0x10582eef0>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15,

In [12]:
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

print(len(train_data))
print(len(val_data))

1003854
111540


In [14]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [40]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f"when the input is:{context}, the target is: {target}.")

when the input is:tensor([18]), the target is: 47.
when the input is:tensor([18, 47]), the target is: 56.
when the input is:tensor([18, 47, 56]), the target is: 57.
when the input is:tensor([18, 47, 56, 57]), the target is: 58.
when the input is:tensor([18, 47, 56, 57, 58]), the target is: 1.
when the input is:tensor([18, 47, 56, 57, 58,  1]), the target is: 15.
when the input is:tensor([18, 47, 56, 57, 58,  1, 15]), the target is: 47.
when the input is:tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is: 58.


In [60]:
torch.manual_seed(42)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    # idx = torch.randint(len(data) - block_size, (batch_size,)) 
    idx = torch.randperm(len(data) - block_size)[:batch_size] # as the random starting point of each block without repeating
    # print(f"initial position of each block is: {idx}")
    x = torch.stack([data[i:i + block_size] for i in idx])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in idx])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-' * 50)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t + 1]
        target = yb[b, t]
        print(f"when the input is:{context}, the target is: {target}.")
    



inputs:
torch.Size([4, 8])
tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [56, 57, 11,  1, 61, 46, 53,  6],
        [36, 18, 27, 30, 16, 10,  0, 15],
        [45, 53, 52, 59, 57,  6,  1, 58]])
targets:
torch.Size([4, 8])
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [57, 11,  1, 61, 46, 53,  6,  1],
        [18, 27, 30, 16, 10,  0, 15, 39],
        [53, 52, 59, 57,  6,  1, 58, 46]])
--------------------------------------------------
when the input is:tensor([57]), the target is: 1.
when the input is:tensor([57,  1]), the target is: 46.
when the input is:tensor([57,  1, 46]), the target is: 47.
when the input is:tensor([57,  1, 46, 47]), the target is: 57.
when the input is:tensor([57,  1, 46, 47, 57]), the target is: 1.
when the input is:tensor([57,  1, 46, 47, 57,  1]), the target is: 50.
when the input is:tensor([57,  1, 46, 47, 57,  1, 50]), the target is: 53.
when the input is:tensor([57,  1, 46, 47, 57,  1, 50, 53]), the target is: 60.
when the input is:tensor([56]), the ta

In [42]:
print(xb) # our input to the transformer

tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [56, 57, 11,  1, 61, 46, 53,  6],
        [36, 18, 27, 30, 16, 10,  0, 15],
        [45, 53, 52, 59, 57,  6,  1, 58]])


In [115]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        # 这里的本质上是做了一个 embedding 初始化的作用，我们默认 nn.Embedding(num_embeddings, embedding_dim)会做随机的初始化，
        # 这里之所以会选择 num_embeddings == embedding_dim == vocab_size， 本质上是我们只是用了一个最简单的情况，
        # 直接用 embedding i 的第 j 个 element 表示 vocab i 对于 j 的预测概率, 直接成为了 logits ，i.e., self.token_embedding_table(i)[j]
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) 
        # (B,T,C), as batch_size = B = 4, block_size = T = 8 这里的 blcok 也可以想象成 time step, channel_size = C = 65


        # 根据 torch.nn.CrossEntropyLoss，https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
        # CEloss 的 input 形式应该是 (minibatch,C)，C 代表的是不同的 class, aka channel_size，在我们的例子里，
        # 因为每一个 batch 实际上都会 block_size 个下一个数字需要预测， 所以 minibatch = 4 * 8，同时因为 logits 的 shape 是 (B,T,C)，
        # 无法直接使用 cross_entropy, 所以我们需要重新改变他们的形状，使得 F.cross_entropy() 的 input 的形状变为
        # （B * T, C）, 和 （B * T）

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        # 逐步通过 logits 的概率生成 max_new_tokens 个新的 tokens
        
        for _ in range(max_new_tokens):
            # get the predictions
            # self(x)（即将模型对象作为一个函数调用）会调用模型的 forward 方法，这是 PyTorch 对 nn.Module 的一个特殊设计。
 
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)，只用最后一个时间步数，也就是 block 的最后一位来进行预测
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)，在最后一个维度 channel 上做 softmax
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)


print(type(m.token_embedding_table))
print(m.token_embedding_table)


print(logits.shape)
print(logits)
print(loss)  
# 根据单个样本的 CEloss = - sum y*log(p) = - log(p), 如果 p 是几乎随机的 1/65， 那么 loss = -log(1/65) = log(65) = 4.174
# 与初始计算的 loss = tensor(4.6168, grad_fn=<NllLossBackward0>)，很接近，说明一开始几乎是随机的

# 从初始的 0 tensor 开始，逐步生成长度最多为 100 的 输出
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


<class 'torch.nn.modules.sparse.Embedding'>
Embedding(65, 65)
torch.Size([32, 65])
tensor([[ 0.4224, -0.8596,  0.2910,  ...,  0.2380, -0.1270,  0.0534],
        [ 0.8564,  2.2181,  0.5232,  ...,  0.4114,  1.9312,  1.0119],
        [ 0.4851,  0.0060,  1.0007,  ..., -0.0776, -0.5015, -2.2270],
        ...,
        [-0.6013, -0.0996, -0.9852,  ..., -0.4292, -0.9818, -0.6421],
        [ 0.8564,  2.2181,  0.5232,  ...,  0.4114,  1.9312,  1.0119],
        [ 1.0923, -1.3024,  1.3521,  ..., -0.2658, -1.4075, -0.7460]],
       grad_fn=<ViewBackward0>)
tensor(4.6168, grad_fn=<NllLossBackward0>)

o$,q&IWqW&xtCjaB?ij&bYRGkF?b; f ,CbwhtERCIfuWr,DzJERjhLlVaF&EjffPHDFcNoGIG'&$qXisWTkJPw
 ,b Xgx?D3sj


In [54]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [116]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # 训练过程
    # evaluate the loss
    logits, loss = m(xb, yb)              # 1. 计算 loss
    optimizer.zero_grad(set_to_none=True) # 2. 将模型参数的梯度清零，防止上一轮的梯度影响当前轮的计算。
    loss.backward()                       # 3. 计算梯度
    optimizer.step()                      # 4. 更新 weight

print(loss.item())


4.657245635986328


In [64]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Tht thak sg any ou,

T:
Willl hoime abut'RDIOMstidsuc thio brans ga, min t it,
Kntonot id maprgor s!

Asou es,
FI; be Hareanotofou DI y:

Crens hikeveshe, courerthant ngaise s by mero, d.
Brere.
OMenend s, fe l! wime y gturdomodd hurthe wo ads p d byo LESHad.
Mingrmyo itu wod
ORUNUnd ms! UCE:
Garee r nd balBetin yes aiders ager n, reXOHe ha.
SCanort  r
Thorwinouliseljowh iak'd, baveay tin.
NThe mitispel de me.
the werocares l wo-d ppate bothived boudors d falsth at y,

fare, aineanfathyofie fath


## The mathematical trick in self-attention

---



In [66]:
# consider the following toy example:

torch.manual_seed(42)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [80]:
# Version 1: 手动计算 up to t 的 bag of words (bow)，本质是计算前 t 项的
# sum pooling。比较麻烦，时间复杂度 O(BT^2C)，额外空间复杂度 O(TC)

# We want each xbow[b, t] to be the mean of x[b, i] for all i <= t, aka x[b, t] = mean_{i<=t} x[b, i]
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t + 1]  # (t + 1, C)
        xbow[b, t] = torch.mean(xprev, 0)

In [87]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
# 利用下三角矩阵快速得到每个index i 的 [:i + 1] 的平均值
torch.manual_seed(21)
a = torch.tril(torch.ones(3, 3))


# print('case1', torch.sum(a, 1).shape)
# print('case2', torch.sum(a, 1, keepdim=True).shape)
# a = a / torch.sum(a, 1)
# 记住不能犯这个错误，一定要加上 keepdim=True 否则除法的时候不对

a = a / torch.sum(a, 1, keepdim=True)


b = torch.randint(0, 10,(3, 2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')

# c 就是前 i 个的 sumpooling
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[3., 3.],
        [6., 8.],
        [6., 4.]])
--
c=
tensor([[3.0000, 3.0000],
        [4.5000, 5.5000],
        [5.0000, 5.0000]])


In [88]:
# version 2: using matrix multiply for a weighted aggregation

wei = torch.tril(torch.ones(T, T))

# 这里的 wei 代表就是之前的 tensor a，可以看到在简单的 sumpooling 模型中，wei 代表之前的所有字符对当前的影响是一样的，不论多远，这明显是不合理的
# 之后的 self attention 会进行修改，通过 QKV 的方式计算不同之前的 char 对于之后的影响

wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T, T) @ (B, T, C) 利用广播机制 ----> (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [86]:
# version 3: using Softmax
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x 
torch.allclose(xbow, xbow3)

True

In [96]:
# version 4: self-attention!
# self-attention 本质是一种 token communication mechanism, 它实现了特定的通过线性计算 token 之前所有的点与 token 的 attention 分数，然后计算 weighted sum

torch.manual_seed(42)
B,T,C = 4,8,32 # batch, time, channels 
# 这里需要注意的是，这里的 channels 不再代指 65 的 vocal_size, 这里我们可以规定 token embedding 的维度，然后通过更改为

# def __init__(self):
#     self.token_embedding_table = nn.Embedding(vocab_size, n_embd),  # c == n_embd
#     self.position_embedding_table = nn.Embedding(block_size, n_embd)  # 这里使用的是 learnable 的 posiiton embedding，从 0 到 T - 1，每一个位置学习一个 embedding
    
#     # 这里是 Transformer 原文的正余弦 position embedding
#     # def sinusoidal_positional_encoding(seq_len, dim):
#     #     pos = torch.arange(seq_len).unsqueeze(1)  # (seq_len, 1)
#     #     div_term = torch.exp(torch.arange(0, dim, 2) * -(math.log(10000.0) / dim))
#     #     pe = torch.zeros(seq_len, dim)
#     #     pe[:, 0::2] = torch.sin(pos * div_term)
#     #     pe[:, 1::2] = torch.cos(pos * div_term)
#     #     return pe

#     # self.position_embedding_table = sinusoidal_positional_encoding(block_size, n_embd)
    
#     self.lm_head = nn.Linear(n_embd, vocab_size) 这里是输出层，因为最后一定是要转换为预测词表的概率，所以一定需要从隐含层 n_embd 映射到 vocab_size

# .
# .
# .
# def forward(self, idx, target=None):
#     tok_emb = self.token_embedding_table(idx)  # (B, T, C)
#     pos_emb = self.position_embedding_table(torch.arange(T, device=device))  #(T, C)
#     x = tok_emb + pos_emb  #(B, T, C)
#     logits = self.lm_head(x)



x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16  # 对于每一个单头的注意力机制，这里 head_size 指的就是将 token 的信息 （tok_embd + pos_embd） -> C 之后，C 再压缩到低维空间的维度
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

# most import in self attention
k = key(x)   # (B, T, 16) # !!!! what do I contain
q = query(x) # (B, T, 16) # !!!! what I'm looking for

wei =  q @ k.transpose(-2, -1) # transpose 最后两个维度 (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)  # !!!! weighted sum of 之前的所有 t 对于我当前 index 的影响
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [95]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1905, 0.8095, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3742, 0.0568, 0.5690, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1288, 0.3380, 0.1376, 0.3956, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4311, 0.0841, 0.0582, 0.3049, 0.1217, 0.0000, 0.0000, 0.0000],
        [0.0537, 0.3205, 0.0694, 0.2404, 0.2568, 0.0592, 0.0000, 0.0000],
        [0.3396, 0.0149, 0.5165, 0.0180, 0.0658, 0.0080, 0.0373, 0.0000],
        [0.0165, 0.0375, 0.0144, 0.1120, 0.0332, 0.4069, 0.3136, 0.0660]],
       grad_fn=<SelectBackward0>)

Notes: 
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [107]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei_1 = q @ k.transpose(-2, -1)
wei_2 = q @ k.transpose(-2, -1) * head_size**-0.5

print(k.var())
print(q.var())

print('-' * 50)
print(wei_1.var())
print(wei_2.var())

print('-' * 50)
print("Softmax may sharpen the distribution and converge to one-hot in case of extreme logits")


test_tensor = torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])

for multipler in range(6):
    print(torch.softmax(test_tensor * 2 ** multipler, -1))



tensor(0.9806)
tensor(0.9566)
--------------------------------------------------
tensor(14.4670)
tensor(0.9042)
--------------------------------------------------
Softmax may sharpen the distribution and converge to one-hot in case of extreme logits
tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])
tensor([0.1720, 0.0944, 0.2565, 0.0944, 0.3827])
tensor([0.1139, 0.0343, 0.2535, 0.0343, 0.5641])
tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])
tensor([1.5939e-03, 1.3117e-05, 3.9102e-02, 1.3117e-05, 9.5928e-01])
tensor([2.7562e-06, 1.8667e-10, 1.6588e-03, 1.8667e-10, 9.9834e-01])


In [112]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



In [117]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5091, val loss 2.5060
step 300: train loss 2.4199, val loss 2.4338
step 400: train loss 2.3499, val loss 2.3561
step 500: train loss 2.2965, val loss 2.3129
step 600: train loss 2.2405, val loss 2.2495
step 700: train loss 2.2053, val loss 2.2194
step 800: train loss 2.1625, val loss 2.1853
step 900: train loss 2.1248, val loss 2.1513
step 1000: train loss 2.1035, val loss 2.1305
step 1100: train loss 2.0709, val loss 2.1193
step 1200: train loss 2.0375, val loss 2.0789
step 1300: train loss 2.0246, val loss 2.0646
step 1400: train loss 1.9936, val loss 2.0371
step 1500: train loss 1.9720, val loss 2.0318
step 1600: train loss 1.9629, val loss 2.0479
step 1700: train loss 1.9422, val loss 2.0139
step 1800: train loss 1.9096, val loss 1.9963
step 1900: train loss 1.9080, val loss 1.9884
step 2000: train loss 1.8837, val loss 1.9943
step 2100: train loss 1.